In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

import re
import string

In [7]:
dfake = pd.read_csv('./dataset/Fake.csv')
dtrue = pd.read_csv('./dataset/True.csv')

In [9]:
dfake['class'] = 0
dtrue['class'] = 1

# Identify the quantity of entries and its columns
dfake.shape, dtrue.shape

((23481, 5), (21417, 5))